
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/monacofj/moeabench/blob/main/examples/example-06.ipynb)

# Example 06: Statistical Hypothesis Testing

This example demonstrates how to perform rigorous statistical comparisons between two algorithms using non-parametric tests and effect size measures.

In [ ]:
!pip install --quiet git+https://github.com/monacofj/moeabench.git


In [ ]:
from MoeaBench import mb

# 1. Setup: Compare NSGA-III and SPEA2 with 10 repetitions
repeats = 10
pop_size = 100
gens = 50
    
exp1 = mb.experiment()
exp1.name = "NSGA-III"
exp1.mop = mb.mops.DTLZ2(M=3)
exp1.moea = mb.moeas.NSGA3(population=pop_size, generations=gens)

exp2 = mb.experiment()
exp2.name = "SPEA2"
exp2.mop = mb.mops.DTLZ2(M=3)
exp2.moea = mb.moeas.SPEA2(population=pop_size, generations=gens)

print(f"Running {repeats} repetitions for each algorithm...")
exp1.run(repeat=repeats)
exp2.run(repeat=repeats)

In [ ]:
# 2. Statistical Inference
print("\n--- Inferential Statistics ---")
    
# res1 contains:
#             .statistic       test statistic (U)
#             .p_value         probability of observing the data by chance
#             .significant     boolean (p < 0.05)
#             .report()        narrative summary
res1 = mb.stats.mann_whitney(exp1, exp2)
print(res1.report())

In [ ]:
# res2 contains:
#             .statistic       KS distance (D)
#             .p_value         probability of distribution similarity
#             .report()        narrative summary
res2 = mb.stats.ks_test(exp1, exp2)
print("\n" + res2.report())
        
# res3 contains:
#             .value           Vargha-Delaney A12 effect size [0, 1]
#             .report()        narrative summary
res3 = mb.stats.a12(exp1, exp2)
print("\n" + res3.report())

In [ ]:
# 3. New Perspectives (Visualization - mb.view)
print("\n--- Scientific Perspectives (mb.view) ---")
    
# Caste Perspective (Floating quality/density)
print("Generating Caste Profile...")
mb.view.casteplot(exp1, exp2)

# Structural Perspective (Selection pressure)
print("Generating Rank Structure...")
mb.view.rankplot(exp1, exp2)
    
# Competitive Perspective (Tier Duel)
print("Generating Tier Duel (Competitive Stratification)...")
res_tier = mb.stats.tier(exp1, exp2)
print(res_tier.report())
mb.view.tierplot(res_tier)

### Interpretation

Statistical tests avoid the trap of "visual optimization." The Mann-Whitney U test tells us if one algorithm is significantly better than the other based on the median performance.

In MoeaBench, these tests are "smart": they automatically handle the extraction of metrics (like Hypervolume) and set a common reference point.

**Scientific Perspectives (mb.view)**:
1. **Caste Plot**: Visualizes Quality vs Density per dominance rank.
2. **Rank Plot**: Visualizes the global frequency of each rank (selection pressure).
3. **Tier Plot**: Shows the competitive proportions of each algorithm per rank.